In [1]:
use Modern::Perl;
use HackaMol;
use Math::Vector::Real;

# Benzene is more than 12 vectors: using HackaMol to work with atomic "metadata" and coordinates.
Suggested Prerequisites:

1. This notebook builds directly from the [p5-Vectors-Benzene notebook](https://github.com/demianriccardi/EC-Chem452/blob/master/Notebooks/p5-Vectors-Benzene_2016_02_26.ipynb). 

2. This notebook uses the HackaMol library.  Please review the [Readme](https://github.com/demianriccardi/HackaMol) and the linked [HackaMol paper](https://www.researchgate.net/profile/Demian_Riccardi/publication/273778191_HackaMol_an_object-oriented_Modern_Perl_library_for_molecular_hacking_on_multiple_scales/links/550ebec60cf27526109e6ade.pdf) for additional information about the library.


### Ability to work with vectors while maintaining a connection to the atomic metadata is prerequisite to carrying out structural analysis of biological molecules.
At a given temperature, pressure, and molar constitution $G(T,P,n_1,n_2 ... )$, a biological molecule (or collection of molecules) has a space of available conformations or configurations that may be large ($S=k\ln W$).  If you were to carry out a molecular dynamics simulation (which we will come back to later), you would most likely prepare your system using a single structure provided by X-ray crystallography or NMR spectroscopy, solvate your system in water (maybe with salt ions), and then add some heat so that it wiggles about.  

When simulated in a computer, a condensed phase system is just a vector space that varies with time (molecular dynamics) or conformation (monte carlo) that is restrained (or even constrained) by the information of how suppose that the vectors interact (e.g. this water hydrogen, with these coordinates, is bound to this water oxygen with these coordinates). While this may seem an obfuscated way to describe computational work with molecules, it is a valuable perspective.  The hierarchy of chemistry and biochemistry (atom, atoms, bonds, molecule, molecules, polymers, nonbonded interactions, etc.) provide the information that biases that which is possible in empirical computational work (i.e. classical mechanical molecular simulations). 

#### Lab exercise: Fun with benzene atoms and coordinates using vector analysis
I wrote the HackaMol library to make my computational life easier, more fun, more sophisticated, and more reproducible.  HackaMol provides a molecular object system that packages atomic and molecular data into classes that have methods for interacting with the data.  In the previous exercise, we stripped benzene down to nothing but a a collection of 12 vectors.  Now we will add atomic information back to these vectors.  We'll call this information "metadata"; metadata is data that describes other data.  We can add such information to those 12 vectors to name atoms, add bonds, angles, and dihedrals all with names and information. We will do that here.   

In this lab we will use benzene to interact with the hierarchy [HackaMol](https://metacpan.org/pod/HackaMol) classes: HackaMol::Atom (https://metacpan.org/pod/HackaMol::Atom), [HackaMol::AtomGroup](https://metacpan.org/pod/HackaMol::AtomGroup), [HackaMol::Bond](https://metacpan.org/pod/HackaMol::Bond), [HackaMol::Angle](https://metacpan.org/pod/HackaMol::Angle), [HackaMol::Dihedral](https://metacpan.org/pod/HackaMol::Dihedral), and [HackaMol::Molecule](https://metacpan.org/pod/HackaMol::Molecule). The HackaMol::Atom objects contain coordinates which are just Math::Vector::Real objects;  the other classes listed above contain groups of HackaMol::Atom objects plus other things.  

#### Mental exercise: 
Before going an further.  Again, take a look at the PDB file for a mutant of T4 Lysozyme [PDBID: 1L54](http://www.rcsb.org/pdb/files/1L54.pdb). Find the lines of coordinates that also contain additional metadata. What are all the different types of metadata that you can see in this file?


### Let's define a string (c6h6_xyz) that contains the xyz file description of a bdenzene molecule:

In [2]:


my $c6h6_xyz =
'  C        0.00000        1.40272        0.00000
  H        0.00000        2.49029        0.00000
  C       -1.21479        0.70136        0.00000
  H       -2.15666        1.24515        0.00000
  C       -1.21479       -0.70136        0.00000
  H       -2.15666       -1.24515        0.00000
  C        0.00000       -1.40272        0.00000
  H        0.00000       -2.49029        0.00000
  C        1.21479       -0.70136        0.00000
  H        2.15666       -1.24515        0.00000
  C        1.21479        0.70136        0.00000
  H        2.15666        1.24515        0.00000
'; 

print $c6h6_xyz;


  C        0.00000        1.40272        0.00000
  H        0.00000        2.49029        0.00000
  C       -1.21479        0.70136        0.00000
  H       -2.15666        1.24515        0.00000
  C       -1.21479       -0.70136        0.00000
  H       -2.15666       -1.24515        0.00000
  C        0.00000       -1.40272        0.00000
  H        0.00000       -2.49029        0.00000
  C        1.21479       -0.70136        0.00000
  H        2.15666       -1.24515        0.00000
  C        1.21479        0.70136        0.00000
  H        2.15666        1.24515        0.00000


1


### Load this xyz file string into two HackaMol::Molecule objects called bnz1 and bnz2:

In [4]:
my $bnz1 = HackaMol->new->read_string_mol($c6h6_xyz, 'xyz');
my $bnz2 = HackaMol->new->read_string_mol($c6h6_xyz, 'xyz');

$_->name("Benzene") foreach ($bnz1,$bnz2);  # name each of them benzene

say $bnz2->name;

Benzene


1


Both \$bnz1 and \$bnz2 are HackaMol::Molecule objects that have some data and methods for interacting with the data.  While both molecules have all the same values, they are independent of one another.  Currently we have two identical, unique copies of the data packaged in HackaMol::Molecule objects.   Let's dump \$bnz1 out to take a look at it's guts:  

In [5]:
print $bnz1->dump;

$VAR1 = bless( {
                 't' => 0,
                 'bonds' => [],
                 'name' => 'Benzene',
                 'atoms' => [
                              bless( {
                                       'iatom' => 0,
                                       't' => 0,
                                       'symbol' => 'C',
                                       'coords' => [
                                                     bless( [
                                                              '0',
                                                              '1.40272',
                                                              '0'
                                                            ], 'Math::Vector::Real' )
                                                   ],
                                       'name' => 'at0',
                                       'bond_count' => 0
                                     }, 'HackaMol::Atom' ),
                         

1


Looking at the dumped information, you can see that the HackaMol::Molecue has a collection of atoms, each of which is a HackaMol::Atom object, and each atom has coordinates, which are just Math::Vector::Real objects.  There are some other attributes that you will learn about as you progress.  Let's pull out the coordinates and reproduce a little code from the last notebook.

In [6]:
my @vecs = map {$_->xyz} $bnz1->all_atoms;

{0, 1.40272, 0}{0, 2.49029, 0}{-1.21479, 0.70136, 0}{-2.15666, 1.24515, 0}{-1.21479, -0.70136, 0}{-2.15666, -1.24515, 0}{0, -1.40272, 0}{0, -2.49029, 0}{1.21479, -0.70136, 0}{2.15666, -1.24515, 0}{1.21479, 0.70136, 0}{2.15666, 1.24515, 0}


In [7]:
my $count = 0;
foreach my $iv (0 .. $#vecs){
    foreach my $jv ($iv+1 .. $#vecs){
        $count++;
        printf("%5i%5i%10.3f\n",$iv, $jv, abs($vecs[$iv]-$vecs[$jv]));
    }
}
say $count;

    0    1     1.088
    0    2     1.403
    0    3     2.162
    0    4     2.430
    0    5     3.415
    0    6     2.805
    0    7     3.893
    0    8     2.430
    0    9     3.415
    0   10     1.403
    0   11     2.162
    1    2     2.162
    1    3     2.490
    1    4     3.415
    1    5     4.313
    1    6     3.893
    1    7     4.981
    1    8     3.415
    1    9     4.313
    1   10     2.162
    1   11     2.490
    2    3     1.088
    2    4     1.403
    2    5     2.162
    2    6     2.430
    2    7     3.415
    2    8     2.805
    2    9     3.893
    2   10     2.430
    2   11     3.415
    3    4     2.162
    3    5     2.490
    3    6     3.415
    3    7     4.313
    3    8     3.893
    3    9     4.981
    3   10     3.415
    3   11     4.313
    4    5     1.088
    4    6     1.403
    4    7     2.162
    4    8     2.430
    4    9     3.415
    4   10     2.805
    4   11     3.893
    5    6     2.162
    5    7     2.490
    5    8   

1


### Exercise. The HackaMol molecule has some methods that are useful for interacting with the atomic data.

Here is a list of methods to try here:
    1. print_xyz
    2. print_pdb
    3. COM
    4. total_mass
    5. all_atoms
    6. get_atoms(0) to get the first atom
 
Each can be invoked as follows:

$benzene->method

### Exercise. Use the translate method to translate your molecule along the z-axis.  Use the print_xyz method to see the result and then translate it back so that the center of mass is at (0,0,0);

\$mol->translate(-1 * $mol->COM);


### Exercise. Create a line of 21 mercury atoms perpendicular to the plane of the benzene molecule, spanning 10 angstroms above and below the plane of the molecule.

In [5]:
my $vec1 = $benzene->get_atoms(0)->xyz - $benzene->COM ;  # vector to COM
my $vec2 = $benzene->get_atoms(2)->xyz - $benzene->COM ;  # vector to COM

my $perp_vec = ($vec1 x $vec2) -> versor;
 
$benzene->push_atoms( HackaMol::Atom->new(symbol => 'Hg', coords => [$_*0.5*$perp_vec ] ) ) foreach -10 .. 10;

In [6]:
$benzene->print_xyz; 1;

33

  C   0.000000   1.402720   0.000000
  H   0.000000   2.490290   0.000000
  C  -1.214790   0.701360   0.000000
  H  -2.156660   1.245150   0.000000
  C  -1.214790  -0.701360   0.000000
  H  -2.156660  -1.245150   0.000000
  C   0.000000  -1.402720   0.000000
  H   0.000000  -2.490290   0.000000
  C   1.214790  -0.701360   0.000000
  H   2.156660  -1.245150   0.000000
  C   1.214790   0.701360   0.000000
  H   2.156660   1.245150   0.000000
 Hg  -0.000000   0.000000  -5.000000
 Hg  -0.000000   0.000000  -4.500000
 Hg  -0.000000   0.000000  -4.000000
 Hg  -0.000000   0.000000  -3.500000
 Hg  -0.000000   0.000000  -3.000000
 Hg  -0.000000   0.000000  -2.500000
 Hg  -0.000000   0.000000  -2.000000
 Hg  -0.000000   0.000000  -1.500000
 Hg  -0.000000   0.000000  -1.000000
 Hg  -0.000000   0.000000  -0.500000
 Hg   0.000000  -0.000000   0.000000
 Hg   0.000000  -0.000000   0.500000
 Hg   0.000000  -0.000000   1.000000
 Hg   0.000000  -0.000000   1.500000
 Hg   0.000000  -0.000000   2.0000

1
